In [1]:
%matplotlib inline
# importar biblioteca Pandas
import pandas as pd

# importar Matplotlib biblioteca de gráficos
import matplotlib.pyplot as plt

# importar Seaborn biblioteca de gráficos
import seaborn as sns

import geopandas as gpd

# Importar Numpy, train_test_split
import numpy as np
from sklearn.model_selection import train_test_split

# importa o modelo de regressão linear, metrics, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import metrics

from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import LabelEncoder

# acesso ao so e ao sys
import os
import sys

# Garbage collector, para recuperar memória após carregar o df inicial
import gc

import graphviz.backend as be
import statistics as sts

# Importar algoritmo
# from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from mpl_toolkits.mplot3d import axes3d

# import dtreeviz

random_state = 1234

# controlar as notificações
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.float_format", lambda x: "%.8f" % x)

%config InlineBackend.figure_format = 'retina' # Make visualizations look good
# %config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [2]:
url = "./faccionados_analise_vinculo.csv"
df = pd.read_csv(url)

In [3]:
print(list(df.columns))
len(df.columns)

['Unnamed: 0', 'foto_recente', 'fotos_tatuagens', 'nome_completo', 'vulgo_alcunha', 'filiacao', 'data_nascimento', 'cpf', 'rg', 'orgao_expedidor', 'cnh', 'uf_cnh', 'faccao_funcao', 'area_atuacao', 'area_que_ja_atuou', 'bos', 'registro_como_reeducando', 'período_como_reeducando', 'situacao_juridica', 'alvo_sspal', 'qual_operacao_foi_alvo', 'ocorrencias_destaque', 'pena_presidio_federal', 'qual_presidio_estado', 'endereco_atual', 'endereco_anterior', 'endereco_familiar', 'cep', 'latitude_cadastro', 'logitude_cadastro', 'tornozeleira', 'redes_sociais', 'registro_homizio', 'informacoes_homizidio', 'parentes_crime', 'informacoes_parentes_criminosos', 'nome_completo_conjugue', 'tempo_conjugue', 'tipos_bens', 'descricao_bens', 'observacoes', 'faccao', 'bairro_atual', 'cidade_atual', 'uf_atual', 'tipo_rede_social', 'historico', 'latitude_atual', 'longitude_atual']


49

In [4]:
df.head(5)

,Unnamed: 0,foto_recente,fotos_tatuagens,nome_completo,vulgo_alcunha,filiacao,data_nascimento,cpf,rg,orgao_expedidor,cnh,uf_cnh,faccao_funcao,area_atuacao,area_que_ja_atuou,bos,registro_como_reeducando,período_como_reeducando,situacao_juridica,alvo_sspal,qual_operacao_foi_alvo,ocorrencias_destaque,pena_presidio_federal,qual_presidio_estado,endereco_atual,endereco_anterior,endereco_familiar,cep,latitude_cadastro,logitude_cadastro,tornozeleira,redes_sociais,registro_homizio,informacoes_homizidio,parentes_crime,informacoes_parentes_criminosos,nome_completo_conjugue,tempo_conjugue,tipos_bens,descricao_bens,observacoes,faccao,bairro_atual,cidade_atual,uf_atual,tipo_rede_social,historico,latitude_atual,longitude_atual
0,1,https://drive.google.com/open?id=1s1cpbkt6wqv1...,NaN,KAYO NASCIMENTO DE MAGALHÃES,CABRA/ 99,MICHELINE GOMES DE MAGALHÃES E ANTÔNIO SÉRGIO ...,08/01/2000,12514121493.00000000,40840859,SSPAL,NaN,NaN,FINANCEIRO,NaN,JARBAS OITICICA (RIO LARGO) E ACAMPAMENTO DAND...,SUPOSTO AUTOR DE ROUBO - 11442841-00/2015/2704...,SIM,NaN,FORAGIDO,SIM,NaN,NaN,NÃO,NaN,Complexo do Alemão/RJ,NaN,"R. NOSSA SENHORA DA CONCEICAO , 504 , CEP:\n57...",NaN,NaN,NaN,NÃO,NaN,SIM,Informações que está homiziado no Complexo do ...,NÃO,NaN,AMALIA NATALIA DE LIMA SANTOS,5 A 10 ANOS,NaN,NaN,NaN,NI,NaN,NaN,NaN,NaN,Atua no financeiro da cúpula de Alagoas da fac...,-22.86130840,-43.27213680
1,2,https://drive.google.com/open?id=1_KcAyEB2pc7U...,https://drive.google.com/open?id=1-_qEifQW0BVH...,JOSÉ MARIANO DOS SANTOS,RATO,MARIA JOSEFA DOS SANTOS E MÁRIO PEREIRA DOS SA...,20/05/1986,9246079477.00000000,NaN,NaN,NaN,NaN,BRAÇO DIREITO,NaN,NaN,CUMPRIMENTO DE MANDADO DE PRISÃO - 09246079477...,SIM,NaN,FORAGIDO,SIM,NaN,NaN,NaN,NaN,"Rua Yacatan, nº 95, Inhauma, Rio de Janeiro, C...",NaN,NaN,NaN,NaN,NaN,NÃO,NaN,SIM,Informações que encontra-se homiziado no Compl...,SIM,ESPOSA TEM REGISTRO NA SERIS COMO REEDUCANDO,SARA JULICELIA DO NASCIMENTO NOBERTO,15 A 20 ANOS,EMPRESAS,"SARA MAKE UP, 43881885000198, SARA JULICELIA D...",NaN,NI,NaN,NaN,NaN,NaN,Braço direito do líder da facção em Alagoas,-22.86520790,-43.27471140
2,3,https://drive.google.com/open?id=1UmtMexsz9jk3...,https://drive.google.com/open?id=1NMmJTKBYKx--...,GIVALDO BARBOSA DE FRANÇA,QUINZINHO,MARLY ROSA BARBOSA DE FRANÇA E JOSÉ LOPES DE F...,13/04/1981,7275481405.00000000,1999001021061,SSPAL,04110193167,AL,CHEFE/LÍDER,NaN,NaN,SUPOSTO AUTOR DE DIREÇÃO PERIGOSA DE VEÍCULO E...,SIM,NaN,FORAGIDO,SIM,NaN,Suspeito de ameaçar secretário de Segurança Pú...,SIM,NaN,Complexo do Alemão/ RJ,NaN,NaN,NaN,NaN,NaN,NÃO,NaN,SIM,Informações que está homiziado junto com outra...,SIM,"A ESPOSA, MARIA CÉLIA DA SILVA, TEM CADASTRO N...",MARIA CÉLIA DA SILVA,15 A 20 ANOS,VEÍCULOS,NaN,NaN,NI,NaN,NaN,NaN,NaN,"Liderança da facção na parte baixa de Maceió, ...",-22.86130840,-43.27213680
3,4,https://drive.google.com/open?id=1iSR7mQPMKSBk...,NaN,JOSÉ ERIVALDO DA PAZ,ÍNDIO,MARIA HELENA DA PAZ E ANTONIO FERNANDES DA PAZ,19/11/1990,9482616413.00000000,33582840,SSPAL,05310496230,SE,CHEFE/LÍDER,NaN,ARAPIRACA/AL (SENDO ORIUNDO DO BAIRRO PRIMAVER...,SUPOSTO AUTOR DE HOMICÍDIO SIMPLES - 1023524...,SIM,NaN,LIBERADO ATRAVÉS DE ALVARÁ,SIM,NaN,"Acusado de torturar, mutilar e matar o próprio...",NaN,NaN,NaN,"RUA ESMERALDO DA SILVA, 578, CASA, TABOCAS, 49...","R. ROSENDO MAIA , 603 , CEP: 57310-520\nBRASIL...",NaN,NaN,NaN,NÃO,NaN,SIM,Tem endereço da CNH cadastrado no Estado de SE...,SIM,\nO IRMÃO JOSÉ ALDERIVAN DA PAZ TEM BO DE CUMP...,NaN,NaN,NaN,NaN,NaN,NI,NaN,NaN,NaN,NaN,Vice liderança da facção em Arapiraca e respon...,0.00000000,0.00000000
4,5,https://drive.google.com/open?id=1RD6PwugnwRo9...,NaN,LUCAS MANOEL DA SILVA,PROJETO,MARIA MARGARETE CORREIA DA SILVA\nJUVENAL MANO...,26/09/1998,NaN,\t3523498-9,SSPAL,NaN,NaN,SOLDADO DO TRÁFICO,CLIMA BOM/FRONTEIRA,NÃO,"ROUBO,0012-B/18-0508, JOSÉ RAFAEL FERREIRA DA ...",SIM,06/04/2018,SEMIABERTO,NÃO,NaN,NÃO,NÃO,NÃO,"CONJUNTO JARBAS OITICICA,RIO LARGO","RUA NOSSA SENHORA DA CONCEIÇÃO, N°504, CLIMA BOM","RUA N

In [10]:
print(df.groupby("faccao").size())

faccao
CV         75
NEUTRO     11
NI        107
PCC        33
SF          2
dtype: int64


In [6]:
df["facao_label"] = df.faccao.astype("category").cat.codes
df["funcao_label"] = df.faccao_funcao.astype("category").cat.codes

df["bairro_label"] = df.bairro_atual.astype("category").cat.codes
df["cidade_label"] = df.cidade_atual.astype("category").cat.codes

In [7]:
df

,Unnamed: 0,foto_recente,fotos_tatuagens,nome_completo,vulgo_alcunha,filiacao,data_nascimento,cpf,rg,orgao_expedidor,cnh,uf_cnh,faccao_funcao,area_atuacao,area_que_ja_atuou,bos,registro_como_reeducando,período_como_reeducando,situacao_juridica,alvo_sspal,qual_operacao_foi_alvo,ocorrencias_destaque,pena_presidio_federal,qual_presidio_estado,endereco_atual,endereco_anterior,endereco_familiar,cep,latitude_cadastro,logitude_cadastro,tornozeleira,redes_sociais,registro_homizio,informacoes_homizidio,parentes_crime,informacoes_parentes_criminosos,nome_completo_conjugue,tempo_conjugue,tipos_bens,descricao_bens,observacoes,faccao,bairro_atual,cidade_atual,uf_atual,tipo_rede_social,historico,latitude_atual,longitude_atual,facao_label,funcao_label,bairro_label,cidade_label
0,1,https://drive.google.com/open?id=1s1cpbkt6wqv1...,NaN,KAYO NASCIMENTO DE MAGALHÃES,CABRA/ 99,MICHELINE GOMES DE MAGALHÃES E ANTÔNIO SÉRGIO ...,08/01/2000,12514121493.00000000,40840859,SSPAL,NaN,NaN,1,NaN,JARBAS OITICICA (RIO LARGO) E ACAMPAMENTO DAND...,SUPOSTO AUTOR DE ROUBO - 11442841-00/2015/2704...,SIM,NaN,FORAGIDO,SIM,NaN,NaN,NÃO,NaN,Complexo do Alemão/RJ,NaN,"R. NOSSA SENHORA DA CONCEICAO , 504 , CEP:\n57...",NaN,NaN,NaN,NÃO,NaN,SIM,Informações que está homiziado no Complexo do ...,NÃO,NaN,AMALIA NATALIA DE LIMA SANTOS,5 A 10 ANOS,NaN,NaN,NaN,NI,NaN,NaN,NaN,NaN,Atua no financeiro da cúpula de Alagoas da fac...,-22.86130840,-43.27213680,2,3,-1,-1
1,2,https://drive.google.com/open?id=1_KcAyEB2pc7U...,https://drive.google.com/open?id=1-_qEifQW0BVH...,JOSÉ MARIANO DOS SANTOS,RATO,MARIA JOSEFA DOS SANTOS E MÁRIO PEREIRA DOS SA...,20/05/1986,9246079477.00000000,NaN,NaN,NaN,NaN,2,NaN,NaN,CUMPRIMENTO DE MANDADO DE PRISÃO - 09246079477...,SIM,NaN,FORAGIDO,SIM,NaN,NaN,NaN,NaN,"Rua Yacatan, nº 95, Inhauma, Rio de Janeiro, C...",NaN,NaN,NaN,NaN,NaN,NÃO,NaN,SIM,Informações que encontra-se homiziado no Compl...,SIM,ESPOSA TEM REGISTRO NA SERIS COMO REEDUCANDO,SARA JULICELIA DO NASCIMENTO NOBERTO,15 A 20 ANOS,EMPRESAS,"SARA MAKE UP, 43881885000198, SARA JULICELIA D...",NaN,NI,NaN,NaN,NaN,NaN,Braço direito do líder da facção em Alagoas,-22.86520790,-43.27471140,2,1,-1,-1
2,3,https://drive.google.com/open?id=1UmtMexsz9jk3...,https://drive.google.com/open?id=1NMmJTKBYKx--...,GIVALDO BARBOSA DE FRANÇA,QUINZINHO,MARLY ROSA BARBOSA DE FRANÇA E JOSÉ LOPES DE F...,13/04/1981,7275481405.00000000,1999001021061,SSPAL,04110193167,AL,1,NaN,NaN,SUPOSTO AUTOR DE DIREÇÃO PERIGOSA DE VEÍCULO E...,SIM,NaN,FORAGIDO,SIM,NaN,Suspeito de ameaçar secretário de Segurança Pú...,SIM,NaN,Complexo do Alemão/ RJ,NaN,NaN,NaN,NaN,NaN,NÃO,NaN,SIM,Informações que está homiziado junto com outra...,SIM,"A ESPOSA, MARIA CÉLIA DA SILVA, TEM CADASTRO N...",MARIA CÉLIA DA SILVA,15 A 20 ANOS,VEÍCULOS,NaN,NaN,NI,NaN,NaN,NaN,NaN,"Liderança da facção na parte baixa de Maceió, ...",-22.86130840,-43.27213680,2,0,-1,-1
3,4,https://drive.google.com/open?id=1iSR7mQPMKSBk...,NaN,JOSÉ ERIVALDO DA PAZ,ÍNDIO,MARIA HELENA DA PAZ E ANTONIO FERNANDES DA PAZ,19/11/1990,9482616413.00000000,33582840,SSPAL,05310496230,SE,1,NaN,ARAPIRACA/AL (SENDO ORIUNDO DO BAIRRO PRIMAVER...,SUPOSTO AUTOR DE HOMICÍDIO SIMPLES - 1023524...,SIM,NaN,LIBERADO ATRAVÉS DE ALVARÁ,SIM,NaN,"Acusado de torturar, mutilar e matar o próprio...",NaN,NaN,NaN,"RUA ESMERALDO DA SILVA, 578, CASA, TABOCAS, 49...","R. ROSENDO MAIA , 603 , CEP: 57310-520\nBRASIL...",NaN,NaN,NaN,NÃO,NaN,SIM,Tem endereço da CNH cadastrado no Estado de SE...,SIM,\nO IRMÃO JOSÉ ALDERIVAN DA PAZ TEM BO DE CUMP...,NaN,NaN,NaN,NaN,NaN,NI,NaN,NaN,NaN,NaN,Vice liderança da facção em Arapiraca e respon...,0.00000000,0.00000000,2,0,-1,-1
4,5,https://drive.google.com/open?id=1RD6PwugnwRo9...,NaN,LUCAS MANOEL DA SILVA,PROJETO,MARIA MARGARETE CORREIA DA SILVA\nJUVENAL MANO...,26/09/1998,NaN,\t3523498-9,SSPAL,NaN,NaN,3,CLIMA BOM/FRONTEIRA,NÃO,"ROUBO,0012-B/18-0508, JOSÉ RAFAEL FERREIRA DA ...",SIM,06/04/2018,SEMIABERTO,NÃO,NaN,NÃO,NÃO,NÃO,"CONJUNTO JARBAS OITICICA,RIO LARGO","RUA NOSSA SENHORA DA CO

In [5]:
# df.faccao_funcao.replace({"CHEFE/LÍDER": 1}, regex=True, inplace=True)
# df.faccao_funcao.replace({"BRAÇO DIREITO": 2}, regex=True, inplace=True)
# df.faccao_funcao.replace({"SOLDADO DO TRÁFICO": 3}, regex=True, inplace=True)
# df.faccao_funcao.replace({"FINANCEIRO": "1"}, regex=True, inplace=True)
# df.faccao_funcao.replace({"FINANCEIRO": "1"}, regex=True, inplace=True)
# df.faccao_funcao.replace({"FINANCEIRO": "1"}, regex=True, inplace=True)


# df.Q8.replace({"PREDOMINANTEMENTE MENTAL": "2"}, regex=True, inplace=True)
# df.Q8.replace({"MISTA \(FÍSICA E MENTAL\)": "3"}, regex=True, inplace=True)

AttributeError: 'DataFrame' object has no attribute 'Q8'

In [ ]:
# nomes = df[["nome_completo", "vulgo_alcunha", "cpf", "rg", "cnh"]].copy()
# fotos = df[["foto_recente", "fotos_tatuagens"]].copy()
# faccao = df[["faccao"]].copy()
# funcao = df[["faccao_funcao"]].copy()
# area_atuacao = df[["area_atuacao"]].copy()
# ja_atuou = df[["area_que_ja_atuou"]].copy()
# filiacao = df[["filiacao", "data_nascimento"]].copy()
# alvo_sspal = df[["alvo_sspal", "qual_operacao_foi_alvo"]].copy()
# reeducando = df[["registro_como_reeducando", "período_como_reeducando"]].copy()
# conjugue = df[["nome_completo_conjugue", "tempo_conjugue", "uf_atual"]].copy()
# endereco_anterior = df[["endereco_anterior"]].copy()
# endereco_familiar = df[["endereco_familiar"]].copy()
# endereco_atual = df[
#     ["endereco_atual", "bairro_atual", "cidade_atual", "uf_atual"]
# ].copy()
# bairro = df[["bairro_atual"]].copy()
# cidade = df[["cidade_atual"]].copy()
# uf = df[["uf_atual"]].copy()
# cep = df[["cep"]].copy()
# coordenadas = df[["latitude_atual", "longitude_atual"]].copy()
# analises_vinculos = df[
#     [
#         "nome_completo",
#         "vulgo_alcunha",
#         "faccao",
#         "faccao_funcao",
#         "bairro_atual",
#         "cidade_atual",
#         "uf_atual",
#         "area_atuacao",
#     ]
# ].copy()

In [ ]:
path_faccoes = "./"

In [ ]:
# # df.to_csv(path_faccoes + "faccionados.csv", index=True)
# df.to_csv(path_faccoes + "faccionados_neo4j_00.csv", index=True)

# # csvs desmembrados
# nomes.to_csv(path_faccoes + "nomes.csv", index=True)
# fotos.to_csv(path_faccoes + "fotos.csv", index=True)
# faccao.to_csv(path_faccoes + "faccao.csv", index=True)
# funcao.to_csv(path_faccoes + "funcao.csv", index=True)
# area_atuacao.to_csv(path_faccoes + "area_atuacao.csv", index=True)
# ja_atuou.to_csv(path_faccoes + "ja_atuou.csv", index=True)
# filiacao.to_csv(path_faccoes + "filiacao.csv", index=True)
# alvo_sspal.to_csv(path_faccoes + "alvo_sspal.csv", index=True)
# reeducando.to_csv(path_faccoes + "reeducando.csv", index=True)
# conjugue.to_csv(path_faccoes + "conjugue.csv", index=True)
# endereco_anterior.to_csv(path_faccoes + "endereco_anterior.csv", index=True)
# endereco_familiar.to_csv(path_faccoes + "endereco_familiar.csv", index=True)
# endereco_atual.to_csv(path_faccoes + "endereco_atual.csv", index=True)
# bairro.to_csv(path_faccoes + "bairro.csv", index=True)
# cidade.to_csv(path_faccoes + "cidade.csv", index=True)
# uf.to_csv(path_faccoes + "uf.csv", index=True)
# cep.to_csv(path_faccoes + "cep.csv", index=True)
# coordenadas.to_csv(path_faccoes + "coordenadas.csv", index=True)
# coordenadas.to_csv(path_faccoes + "coordenadas.csv", index=True)
# analises_vinculos.to_csv(path_faccoes + "analises_vinculos.csv", index=True)